<a href="https://colab.research.google.com/github/KayvanShah1/usc-dsci553-data-mining-sp24/blob/main/assignment-2/notebooks/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup & Installation

In [1]:
!pip install pyspark ipython-autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 43.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=1e061f485ab579fcab9741b139db33ac8560be21b2c9db1c3f5cdebb56cb33ab
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
%%bash
java --version

openjdk 11.0.21 2023-10-17
OpenJDK Runtime Environment (build 11.0.21+9-post-Ubuntu-0ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.21+9-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)


# Imports

In [3]:
import os
import sys
import json
from pyspark import SparkContext

from pprint import pprint

%load_ext autotime

time: 330 µs (started: 2024-02-24 11:23:05 +00:00)


In [4]:
os.chdir("/content/drive/MyDrive/Colab Notebooks/DSCI553/hw2")
os.getcwd()

'/content/drive/MyDrive/Colab Notebooks/DSCI553/hw2'

time: 1.13 s (started: 2024-02-24 11:23:05 +00:00)


In [5]:
!ls

Assignment_2.ipynb  ta_feng_all_months_merged.csv  task1_ref.py       task1-test-sample.txt
small1.csv	    task1_refactored.py		   task1-small.txt    task1.txt
small2.csv	    task1_refactored-small.txt	   task1-small-x.txt
time: 1.21 s (started: 2024-02-24 11:23:07 +00:00)


# Tasks

## Task 1

In [22]:
import sys
import time
from pyspark import SparkContext, SparkConf

from collections import Counter
from itertools import combinations, chain
import copy


def generate_baskets(data, case):
    """
    Generate baskets based on the specified case.
    """
    if case == 1:
        # Group Business
        baskets = data.map(lambda line: line.split(",")).map(lambda x: (x[0], [x[1]]))
    elif case == 2:
        # Group User
        baskets = data.map(lambda line: line.split(",")).map(lambda x: (x[1], [x[0]]))
    return baskets.reduceByKey(lambda a, b: a + b)


def hash_function(item1, item2, num_buckets):
    """
    Polynomial hash function to distribute items into different buckets.
    """
    hash_val = int(item1) ^ int(item2)
    return hash_val % num_buckets


def PCY(baskets, total_num_baskets, support, num_buckets):
    """
    Run the PCY algorithm to find candidates
    """
    candidates = []
    baskets = list(baskets)

    support_ratio = len(baskets) / total_num_baskets
    partition_support_threshold = support_ratio * support

    # First pass:
    item_counts = Counter()
    pair_counts = Counter()
    hash_counts = Counter()

    bitmap = [0] * num_buckets

    for basket in baskets:
        # Count item pairs
        item_counts.update(basket)

        pairs = list(combinations(basket, 2))
        pair_counts.update(pairs)

        # Hash pairs and count the number of occurence
        hashes = [hash_function(item[0], item[1], num_buckets) for item in pairs]
        hash_counts.update(hashes)

    # Update the bitmap if count exceeds support threshold
    for h, count in hash_counts.items():
        if count >= partition_support_threshold:
            bitmap[h] = 1

    # Second Pass:
    frequent_singles = []
    for item, count in item_counts.items():
        if count >= partition_support_threshold:
            frequent_singles.append(item)
            candidates.append((tuple([item]), 1))

    frequent_pairs = []
    for item in pair_counts.keys():
        hash_val = hash_function(item[0], item[1], num_buckets)
        if bitmap[hash_val] == 1:
            frequent_pairs.append(item)

    if len(frequent_pairs) == 0:
        return []

    # Filter items from basket that are not frequent
    baskets = [[item for item in basket if item in frequent_singles] for basket in baskets]

    # Initialize candidate dictionary
    candidate_dict = {pair: 0 for pair in frequent_pairs}

    # Count occurrences of frequent pairs in baskets
    for basket in baskets:
        for pair in frequent_pairs:
            if set(pair).issubset(basket):
                candidate_dict[pair] += 1

    # Filter the pair that do not meet the support threshold
    candidate_dict = {
        pair: count for pair, count in candidate_dict.items() if count >= partition_support_threshold
    }

    # Append new candidates to the candidates list
    candidates += [(pair, 1) for pair in candidate_dict.keys()]

    # Valid Candidates
    val_candidates = candidate_dict.keys()

    # Check for frequent pairs in larger sets
    k = 3
    while len(val_candidates) > 0:
        unq_singles = set(chain.from_iterable(val_candidates))
        itemsets = list(combinations(unq_singles, k))

        # Initialize candidate dictionary
        candidate_dict = {itemset: 0 for itemset in itemsets}

        # Count occurrences of frequent pairs in baskets
        for basket in baskets:
            for itemset in itemsets:
                if set(itemset).issubset(basket):
                    candidate_dict[itemset] += 1

        # Filter the pair that do not meet the support threshold
        candidate_dict = {
            itemset: count for itemset, count in candidate_dict.items() if count >= partition_support_threshold
        }

        # Append new candidates to the candidates list
        candidates += [(itemset, 1) for itemset in candidate_dict.keys()]

        # Update Valid Candidates
        val_candidates = candidate_dict.keys()

        # Update Order of larger itemset for next iteration
        k += 1

    return candidates


def check_itemsets_in_basket(basket, candidates):
    candidates = list(candidates)
    frequent_itemsets = []

    for basket in baskets:
        for itemset in candidates:
            if set(itemset).issubset(basket):
                frequent_itemsets.append(itemset)

    return frequent_itemsets


def SON(baskets, support, num_buckets=1000):
    candidates_list = []
    frequent_itemsets_list = []

    # Stage 1: Finding Frequent Itemsets using PCY
    num_baskets = baskets.count()
    items = baskets.values()

    candidates = (
        items.mapPartitions(lambda chunk: PCY(chunk, num_baskets, support, num_buckets))
        .reduceByKey(lambda x, y: x + y)
        .map(lambda x: x[0])
        .distinct()
        .sortBy(lambda x: (len(x), tuple(sorted(x[0]))))
    )
    candidates_map = (
        candidates
        .groupBy(lambda x: len(x))
        .collectAsMap()
    )

    pprint(candidates.collect())
    # pprint(candidates.collect())

    # Stage 2: Candidate Pruning
    frequent_itemsets = (
        items.mapPartitions(lambda: chunk: )
    )

    return candidates_list, frequent_itemsets_list


def task1(case_number, support, input_file_path, output_file_path):
    # Initialize Spark
    conf = SparkConf().setAppName("Task 1")
    spark = SparkContext(conf=conf).getOrCreate()
    spark.setLogLevel("ERROR")

    try:
        start_time = time.time()
        # Read the input data
        data = spark.textFile(input_file_path)

        # Drop the header row
        data = data.filter(lambda row: row != 'user_id,business_id')

        # Generate baskets based on the specified case
        baskets = generate_baskets(data, case_number).cache()

        # Run the PCY Algorithm
        candidates, frequent_itemsets = SON(baskets, support, num_buckets=1000)

        # Output the results
        with open(output_file_path, "w") as f:
            f.write(f"Candidates:\n{candidates}\n")
            f.write("Frequent Itemsets:\n")
            for itemset in frequent_itemsets:
                f.write(f"{itemset}\n\n")

        execution_time = time.time() - start_time
        print(f"Duration: {execution_time}\n")

    finally:
        # Stop Spark
        spark.stop()


# if __name__ == "__main__":
#     if len(sys.argv) != 5:
#         print(
#             "Usage: spark-submit --executor-memory 4G --driver-memory 4G "
#             "task1.py <case_number> <support> <input_file_path> <output_file_path>"
#             " --conf spark.driver.extraJavaOptions=-Dlog4j.configuration=log4j.properties"
#         )
#         sys.exit(1)

#     # Read input parameters
#     case_number = int(sys.argv[1])
#     support = int(sys.argv[2])
#     input_file_path = sys.argv[3]
#     output_file_path = sys.argv[4]

#     task1(case_number, support, input_file_path, output_file_path)


task1(1, 2, "small1.csv", "task1-test-sample.txt")

[('100',),
 ('101',),
 ('102',),
 ('103',),
 ('105',),
 ('106',),
 ('107',),
 ('108',),
 ('97',),
 ('98',),
 ('99',),
 ('100', '98'),
 ('100', '102'),
 ('100', '107'),
 ('100', '106'),
 ('100', '101'),
 ('100', '105'),
 ('100', '99'),
 ('100', '103'),
 ('101', '99'),
 ('101', '97'),
 ('101', '102'),
 ('101', '98'),
 ('101', '100'),
 ('102', '98'),
 ('102', '107'),
 ('102', '106'),
 ('102', '103'),
 ('102', '101'),
 ('102', '99'),
 ('102', '97'),
 ('103', '99'),
 ('103', '97'),
 ('103', '105'),
 ('103', '101'),
 ('103', '102'),
 ('103', '106'),
 ('105', '97'),
 ('105', '99'),
 ('105', '103'),
 ('105', '101'),
 ('105', '102'),
 ('105', '107'),
 ('105', '98'),
 ('105', '106'),
 ('106', '101'),
 ('107', '98'),
 ('107', '106'),
 ('107', '99'),
 ('107', '103'),
 ('107', '101'),
 ('108', '100'),
 ('108', '102'),
 ('108', '107'),
 ('108', '98'),
 ('108', '106'),
 ('108', '105'),
 ('108', '99'),
 ('108', '103'),
 ('108', '101'),
 ('97', '99'),
 ('97', '101'),
 ('97', '98'),
 ('97', '100'),
 ('9

### Ref

In [89]:
%%writefile task1_ref.py
from pyspark import SparkContext
from itertools import combinations as comb
import time
import sys
import copy

def hash(x, y, n_bucket):
  return (x * y) % n_bucket

def group_bus(lines):
    return lines.map(lambda row: (row[0], row[1])).groupByKey().mapValues(set).map(lambda row: (row[0], list(row[1])))

def group_user(lines):
    return lines.map(lambda row: (row[1], row[0])).groupByKey().mapValues(set).map(lambda row: (row[0], list(row[1])))

def PCY(baskets, total_n, support, n_bucket):
    baskets = list(baskets)
    item_cnt = {}
    hash_cnt = {}
    bitmap=[0 for x in range(0, n_bucket)]
    candidates = []
    p = len(baskets) / total_n
    t = p * support

    print(p, t)

    #PCY pass1
    for basket in baskets:
        for item in basket:
            if item not in item_cnt.keys():
                item_cnt[item] = 1
            else:
                item_cnt[item] += 1
        for i in range(0, len(basket) - 1):
            for j in range(i + 1, len(basket)):
                idx = hash(int(basket[i]), int(basket[j]), n_bucket)
                if idx not in hash_cnt.keys():
                    hash_cnt[idx] = 1
                else:
                    hash_cnt[idx] += 1
        for key, value in hash_cnt.items():
            if value >= t:
                bitmap[key] = 1

    #PCY pass2
    single_freq = []
    for key, value in item_cnt.items():
        if value >= t:
            single_freq.append(key)
            candidates.append((tuple([key]), 1))
    #single_freq = sorted(single_freq)
    #print(candidates)
    pair_freq = []
    for i in range(0, len(single_freq) - 1):
        for j in range(i + 1, len(single_freq)):
            pair_freq.append((single_freq[i], single_freq[j]))
    for pair in pair_freq:
        if bitmap[hash(int(pair[0]), int(pair[1]), n_bucket)] != 1:
            pair_freq.remove(pair)

    #print(candidates)
    if len(pair_freq) == 0:
        return []
    else:
        baskets_temp = []
        for basket in baskets:
            basket_1 = copy.deepcopy(basket)
            for item in basket:
                if item not in single_freq:
                    basket_1.remove(item)
            baskets_temp.append(basket_1)
        baskets = baskets_temp
        #pair_freq = sorted(pair_freq)
        #print(pair_freq)
        cnt_dict = dict(zip(pair_freq, [0] * len(pair_freq)))
        for basket in baskets:
            for key in cnt_dict.keys():
                if set(list(key)) <= set(basket):
                    cnt_dict[key] += 1
        for key, value in cnt_dict.copy().items():
            if value < t:
                del cnt_dict[key]
            else:
                candidates.append((tuple(key), 1))
        candidate = cnt_dict.keys()
        #print(candidate)
        k = 3
        while(True):
            #candidate = sorted(candidate)
            temp = list(comb(set(a for b in candidate for a in b), k))
            cnt_dict = dict(zip(temp, [0] * len(temp)))
            for basket in baskets:
                for key in cnt_dict.keys():
                    #print(set(list(key)), set(basket))
                    if set(list(key)) <= set(basket):
                        cnt_dict[key] += 1
            #print(cnt_dict, t)
            for key, value in cnt_dict.copy().items():
                if value < t:
                    del cnt_dict[key]
                else:
                    candidates.append((tuple(key), 1))
            candidate = cnt_dict.keys()
            if len(candidate) > 0:
                k += 1
            else:
                break
        #print(candidates)
        return candidates

def SON(baskets, candidates):
    baskets = list(baskets)
    result = {}
    for basket in baskets:
        for items in candidates:
            flag = 0
            for item in items:
                if item not in basket:
                    flag = 1
                    break
            if flag == 0:
                idx = tuple(items)
                if idx not in result.keys():
                    result[idx] = 1
                else:
                    result[idx] += 1
    return result.items()

if __name__ == '__main__':
    case_n = sys.argv[1]
    support = sys.argv[2]
    input_path = sys.argv[3]
    output_path = sys.argv[4]

    n_bucket = 1000
    spark = SparkContext(appName= "task1")
    spark.setLogLevel("ERROR")
    t_s = time.time()
    #spark = SparkSession.builder.enableHiveSupport().getOrCreate()
    #lines = spark.sparkContext.textFile(input_path)
    lines = spark.textFile(input_path)
    #print(lines.collect())
    lines = lines.filter(lambda row: row != 'user_id,business_id').map(lambda row: row.split(','))

    #Case1 or 2
    if int(case_n) == 1:
        baskets = group_bus(lines)
    elif int(case_n) == 2:
        baskets = group_user(lines)

    #SON algorithm stage #1 -> PCY
    #n_partitions = 5
    total_n = baskets.count()
    #baskets = baskets.partitionBy(n_partitions)
    p = baskets.getNumPartitions()
    items = baskets.values()
    a = items.collect()
    candidates = items.mapPartitions(lambda baskets: PCY(baskets, total_n, float(support), n_bucket)).reduceByKey(lambda x, y: x + y).collect()
    #print(candidates)
    result_temp = []
    for i in range(0, len(candidates)):
        result_temp.append(sorted(candidates[i][0]))
    result_temp = sorted(result_temp)
    result_temp = sorted(result_temp, key = lambda x: len(x))
    last = result_temp[-1]
    for i in range(len(result_temp) - 2, -1, -1):
        if last == result_temp[i]:
            del result_temp[i]
        else:
            last = result_temp[i]
    #print(result_temp)
    #Output candidates
    result = 'Candidates:\n'
    l = 1
    for i in range(0, len(result_temp)):
        if len(result_temp[i]) == 1:
            result = result + '(' + str(result_temp[i])[1:-1] + '),'
        else:
            if len(result_temp[i]) != l:
                result = result[:-1] + "\n\n" + '(' + str(result_temp[i])[1:-1] + "),"
                l = len(result_temp[i])
            else:
                result = result + '(' + str(result_temp[i])[1:-1] + "),"
    result = result[:-1] + '\n\n'
    #print(result)

    #SON algorithm stage #2
    freq_items = items.mapPartitions(lambda baskets: SON(baskets, result_temp)).reduceByKey(lambda x, y: x + y).filter(lambda x : x[1] >= int(support)).collect()
    result_temp = []
    for i in range(0, len(freq_items)):
        result_temp.append(sorted(freq_items[i][0]))
    result_temp = sorted(result_temp)
    result_temp = sorted(result_temp, key = lambda x: len(x))
    last = result_temp[-1]
    for i in range(len(result_temp) - 2, -1, -1):
        if last == result_temp[i]:
            del result_temp[i]
        else:
            last = result_temp[i]
    #print(result_temp)
    result += 'Frequent Itemsets:\n'
    l = 1
    for i in range(0, len(result_temp)):
        if len(result_temp[i]) == 1:
            result = result + '(' + str(result_temp[i])[1:-1] + '),'
        else:
            if len(result_temp[i]) != l:
                result = result[:-1] + "\n\n" + '(' + str(result_temp[i])[1:-1] + "),"
                l = len(result_temp[i])
            else:
                result = result + '(' + str(result_temp[i])[1:-1] + "),"
    result = result[:-1]

    with open(output_path, 'w+') as f:
        f.write(result)

    t_e = time.time()

    print("Duration: ", t_e - t_s)

Overwriting task1_ref.py
time: 18.8 ms (started: 2024-02-24 06:44:57 +00:00)


In [91]:
!spark-submit task1_ref.py 1 2 small1.csv task1-small-x.txt --executor-memory 4G --driver-memory 4G

24/02/24 06:45:40 INFO SparkContext: Running Spark version 3.5.0
24/02/24 06:45:40 INFO SparkContext: OS info Linux, 6.1.58+, amd64
24/02/24 06:45:40 INFO SparkContext: Java version 11.0.21
24/02/24 06:45:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/24 06:45:41 INFO ResourceUtils: ==============================================================
24/02/24 06:45:41 INFO ResourceUtils: No custom resources configured for spark.driver.
24/02/24 06:45:41 INFO ResourceUtils: ==============================================================
24/02/24 06:45:41 INFO SparkContext: Submitted application: task1
24/02/24 06:45:41 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount

## Task 2

# THE END